<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/6_deployment_of_agent_module_1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deployment

## Review

We built up to an agent with memory:

* `act` - let the model call specific tools
* `observe` - pass the tool output back to the model
* `reason` - let the model reason about the tool output to decide what to do next (e.g., call another tool or just respond directly)
* `persist state` - use an in memory checkpointer to support long-running conversations with interruptions

## Goals

Now, we'll cover how to actually deploy our agent locally to Studio and to `LangGraph Cloud`.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## Concepts

There are a few central concepts to understand -

`LangGraph` —
- Python and JavaScript library
- Allows creation of agent workflows

[`LangGraph Server`](https://langchain-ai.github.io/langgraph/concepts/langgraph_server/) —
- Bundles the graph code
- Provides a task queue for managing asynchronous operations
- Offers persistence for maintaining state across interactions
- [Deployment options](https://langchain-ai.github.io/langgraph/concepts/deployment_options/) include LangGraph Platform and self-hosted deployment.

`LangGraph Platform` --
- Hosted service for the LangGraph API
- Allows deployment of graphs from GitHub repositories
- Also provides monitoring and tracing for deployed graphs
- Accessible via a unique URL for each deployment

`LangGraph Studio` --
- Integrated Development Environment (IDE) for LangGraph applications
- Uses the API as its back-end, allowing real-time testing and exploration of graphs
- Can be run locally or with cloud-deployment

`LangGraph SDK` --
- Python and JS library for programmatically interacting with LangGraph graphs
- Provides a consistent interface for working with graphs, whether served locally or in the cloud
- Allows creation of clients, access to assistants, thread management, and execution of runs

`RemoteGraph` --
- The RemoteGraph class is a client implementation for calling remote APIs that implement the LangGraph Server API specification.



## Testing Locally

--

**⚠️ DISCLAIMER**

*Running Studio currently requires a Mac. If you are not using a Mac, then use langgraph cli for this step.*

*Also, if you are running this notebook in Collab, run cloudflared or ngrok tunnel to access the API in Collab environment.*

--

We can easily connect with graphs that are served locally in LangGraph Studio!

We do this via the `url` provided in the lower left corner of the Studio UI.

![Screenshot 2024-08-23 at 1.17.05 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4f53080e6802cec34d_deployment%201.png)

### Local Setup Options

1. [Clone Repo](https://github.com/panaversity/learn-agentic-ai-fundamentals)
2. Open [module-1/studio](https://github.com/panaversity/learn-agentic-ai-fundamentals/tree/main/03_langchain_ecosystem/langgraph/course-notebooks/module-1/studio) locally in VS Code and add environment variables.

Now use any of these options suitable for your machine.

#### 1. [LangGraph Studio App](https://langchain-ai.github.io/langgraph/cloud/how-tos/test_local_deployment/) (Available only on Mac OS for now)

#### 2. Run LangGraph Server Locally

- Ensure you have `LANGSMITH_API_KEY` environment variable in .env file.

```bash
pip install langgraph-cli

langgraph up
```

Once setup completes open in bowser:

- Docs: http://127.0.0.1:8123/docs
- [Access Studio](https://langchain-ai.github.io/langgraph/cloud/how-tos/test_local_deployment/#access-studio) in Browser: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:8123


Now continue the following locally or setup a tunnel to use Google Collab Environment:

#### 3. Acces Local URL in Collab using NGROK

In [ ]:
from langgraph_sdk import get_client

In [ ]:
# Replace this with the URL of your own deployed graph
# URL = "http://localhost:8123" # Local Server URL
URL = "https://1da9-116-90-103-46.ngrok-free.app"
client = get_client(url=URL)

# Search all hosted graphs
# assistants = await client.assistants.search()

In [ ]:
assistants = await client.assistants.search()

In [ ]:
assistants

[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'created_at': '2024-11-18T05:14:53.266038+00:00',
  'updated_at': '2024-11-18T05:14:53.266038+00:00',
  'config': {},
  'metadata': {'created_by': 'system'},
  'version': 1,
  'name': 'agent'},
 {'assistant_id': '228f9934-0cdd-5383-92c8-ee8422522cc2',
  'graph_id': 'router',
  'created_at': '2024-11-18T05:14:53.221352+00:00',
  'updated_at': '2024-11-18T05:14:53.221352+00:00',
  'config': {},
  'metadata': {'created_by': 'system'},
  'version': 1,
  'name': 'router'},
 {'assistant_id': '28d99cab-ad6c-5342-aee5-400bd8dc9b8b',
  'graph_id': 'simple_graph',
  'created_at': '2024-11-18T05:14:51.682062+00:00',
  'updated_at': '2024-11-18T05:14:51.682062+00:00',
  'config': {},
  'metadata': {'created_by': 'system'},
  'version': 1,
  'name': 'simple_graph'}]

In [ ]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

In [ ]:
thread

{'thread_id': 'e5b7ffca-25af-4c62-a6c6-e4e904b8b1d3',
 'created_at': '2024-11-22T10:36:59.882489+00:00',
 'updated_at': '2024-11-22T10:36:59.882489+00:00',
 'metadata': {},
 'status': 'idle',
 'config': {},
 'values': None}

Now, we can run our agent [with `client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) with:

* The `thread_id`
* The `graph_id`
* The `input`
* The `stream_mode`

We'll discuss streaming in depth in a future module.

For now, just recognize that we are [streaming](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/) the full value of the state after each step of the graph with `stream_mode="values"`.

The state is captured in the `chunk.data`.

In [ ]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Hi")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="events",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk)

StreamPart(event='events', data={'event': 'on_chain_start', 'data': {'input': {'messages': [{'id': None, 'name': None, 'type': 'human', 'content': 'Hi', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}]}}, 'name': 'agent', 'tags': [], 'run_id': '1efa8bdb-8169-63b8-920c-392d93e91462', 'metadata': {'created_by': 'system', 'run_attempt': 1, 'langgraph_version': '0.2.53', 'langgraph_plan': 'developer', 'langgraph_host': 'self-hosted', 'run_id': '1efa8bdb-8169-63b8-920c-392d93e91462', 'user_id': '', 'graph_id': 'agent', 'thread_id': 'e5b7ffca-25af-4c62-a6c6-e4e904b8b1d3', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca', 'x-forwarded-for': '34.80.234.115', 'x-forwarded-host': '1da9-116-90-103-46.ngrok-free.app', 'x-forwarded-proto': 'https'}, 'parent_ids': []})
StreamPart(event='events', data={'event': 'on_chain_start', 'data': {'input': {'messages': [{'content': 'Hi', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'respon

#### **Note:** We will cover LangGraph Server (Agentic Infrastructure) in detail in the Module 6.

In [ ]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
    thread['thread_id'],
    "agent",
    input=input,
    stream_mode="events",  # Change stream_mode to 'events'
):
    # Check if chunk has data and is not metadata and contains 'messages' key
    if chunk.data and chunk.event != "metadata" and 'messages' in chunk.data:
        # Access the last message in the 'messages' list
        print(chunk.data['messages'][-1])
    else:
        # Handle other event types or log for debugging
        print(f"Received event: {chunk.event} with data: {chunk.data}")

Received event: metadata with data: {'run_id': '1efa8bdb-b59a-67d0-a595-5bc04ddd17ba', 'attempt': 1}
Received event: events with data: {'event': 'on_chain_start', 'data': {'input': {'messages': [{'id': None, 'name': None, 'type': 'human', 'content': 'Multiply 3 by 2.', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}]}}, 'name': 'agent', 'tags': [], 'run_id': '1efa8bdb-b59a-67d0-a595-5bc04ddd17ba', 'metadata': {'graph_id': 'agent', 'created_by': 'system', 'run_attempt': 1, 'langgraph_version': '0.2.53', 'langgraph_plan': 'developer', 'langgraph_host': 'self-hosted', 'run_id': '1efa8bdb-b59a-67d0-a595-5bc04ddd17ba', 'user_id': '', 'thread_id': 'e5b7ffca-25af-4c62-a6c6-e4e904b8b1d3', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca', 'x-forwarded-for': '34.80.234.115', 'x-forwarded-host': '1da9-116-90-103-46.ngrok-free.app', 'x-forwarded-proto': 'https'}, 'parent_ids': []}
Received event: events with data: {'event': 'on_chain_start', 'data': {'input': {'messages':

## Testing with Cloud (Optional)

We can deploy to Cloud via LangSmith, as outlined [here](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud).

### Create a New Repository on GitHub

* Go to your GitHub account
* Click on the "+" icon in the upper-right corner and select `"New repository"`
* Name your repository (e.g., `langchain-academy`)

### Add Your GitHub Repository as a Remote

* Go back to your terminal where you cloned `langchain-academy` at the start of this course
* Add your newly created GitHub repository as a remote

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
* Push to it
```
git push -u origin main
```

### Connect LangSmith to your GitHub Repository

* Go to [LangSmith](hhttps://smith.langchain.com/)
* Click on `deployments` tab on the left LangSmith panel
* Add `+ New Deployment`
* Then, select the Github repository (e.g., `langchain-academy`) that you just created for the course
* Point the `LangGraph API config file` at one of the `studio` directories
* For example, for module-1 use: `module-1/studio/langgraph.json`
* Set your API keys (e.g., you can just copy from your `module-1/studio/.env` file)

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### Work with your deployment

We can then interact with our deployment a few different ways:

* With the [SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk), as before.
* With [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio).

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

To use the SDK here in the notebook, simply ensure that `LANGSMITH_API_KEY` is set!

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("memory-enabled-agent")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://1da9-116-90-103-46.ngrok-free.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [ ]:
# Select the agent
agent = assistants[0]

In [ ]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-11-18T05:14:53.266038+00:00',
 'updated_at': '2024-11-18T05:14:53.266038+00:00',
 'config': {},
 'metadata': {'created_by': 'system'},
 'version': 1,
 'name': 'agent'}

In [ ]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    # Check if 'messages' key exists before accessing it
    if chunk.data and chunk.event != "metadata" and 'messages' in chunk.data:
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8187d11f-df41-4440-92db-8082a1c44d92', 'example': False}
